# Image Processing with Stable Diffusion XL ControlNet
This notebook demonstrates how to process an image using Stable Diffusion XL ControlNet and display the result.

## Step 1: Install Necessary Libraries
First, we need to install the required libraries.

In [ ]:
!pip install transformers diffusers torch pillow

## Step 2: Import Libraries
Next, we import the necessary libraries for the task.

In [ ]:
import torch
from transformers import DPTFeatureExtractor, DPTForDepthEstimation
from diffusers import ControlNetModel, StableDiffusionXLControlNetPipeline, AutoencoderKL
from diffusers.utils import load_image
from PIL import Image
import io
import requests
from google.colab import files

## Step 3: Load Models
Load the pre-trained models for image processing.

In [ ]:
print("Loading models...")
vae = AutoencoderKL.from_pretrained("madebyollin/sdxl-vae-fp16-fix", torch_dtype=torch.float16)
controlnet = ControlNetModel.from_pretrained("diffusers/controlnet-depth-sdxl-1.0", torch_dtype=torch.float16)
pipe = StableDiffusionXLControlNetPipeline.from_pretrained(
    "stabilityai/stable-diffusion-xl-base-1.0",
    controlnet=controlnet,
    vae=vae,
    variant="fp16",
    use_safetensors=True,
    torch_dtype=torch.float16,
)
pipe.enable_model_cpu_offload()
print("Models loaded successfully")

## Step 4: Define the Image Processing Function
Define a function to process the uploaded image using the loaded models.

In [ ]:
def process_image(depth_image_bytes):
    print("Processing image...")
    depth_image = Image.open(io.BytesIO(depth_image_bytes)).convert("RGB")
    controlnet_conditioning_scale = 0.5

    prompt = ""
    images = pipe(
        prompt, image=depth_image, num_inference_steps=30, controlnet_conditioning_scale=controlnet_conditioning_scale,
    ).images
    print("Image processed successfully")
    return images[0]

## Step 5: Upload an Image and Process It
Upload an image file from your local machine and process it using the defined function.

In [ ]:
uploaded = files.upload()

for filename in uploaded.keys():
    with open(filename, "rb") as f:
        depth_image_bytes = f.read()
    output_image = process_image(depth_image_bytes)
    output_image.save("output.jpg")
    print("Output image saved as output.jpg")

## Step 6: Display the Output Image
Display the processed output image.

In [ ]:
output_image.show()